In [20]:
import pandas as pd
import sqlite3
import json
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../app.db")
df = pd.read_sql_query("SELECT * from report_histo", con)
excel = pd.read_excel("Dossier_Patient_clean_myocapt_KC_v6.xlsx")

In [ ]:
my_tree = json.loads(df.loc[0,"ontology_tree"])
def tree_to_dict(my_tree):
    tree_as_dict = {}
    for feature in my_tree:
        tree_as_dict[feature["text"]] = feature["data"].get("presabs")
        print(feature)
tree_to_dict(my_tree)